# 🎬 UVG MAX – Development Mode

---

## 📦 Step 1: Get Code

In [ ]:
# Clone from GitHub
!rm -rf uvg-max-engine
!git clone https://github.com/shya-bubu/uvg-max-engine.git
%cd uvg-max-engine
print("✅ Cloned!")

In [ ]:
# Verify + Install
import sys, os
sys.path.insert(0, os.getcwd())
print(f"📁 {os.getcwd()}")
print("✅ uvg_core" if os.path.exists('uvg_core') else "❌ not found")

!pip install -q python-dotenv requests tqdm torch google-generativeai
!pip install -q azure-cognitiveservices-speech opencv-python Pillow numpy
!pip install -q librosa pyloudnorm moviepy scikit-learn
print("✅ Dependencies installed!")

## 🔑 Step 2: API Keys

In [ ]:
import os

# ============================================
# 🔐 PASTE YOUR API KEYS HERE
# ============================================

os.environ["GEMINI_API_KEY"] = ""  # Required
os.environ["PEXELS_KEY"] = ""       # Required
os.environ["AZURE_SPEECH_KEY"] = "" # Optional
os.environ["AZURE_SPEECH_REGION"] = "eastus"
os.environ["UVG_MOCK_MODE"] = "true"

# ============================================

print("Gemini:", "✅" if os.environ["GEMINI_API_KEY"] else "❌")
print("Pexels:", "✅" if os.environ["PEXELS_KEY"] else "❌")

## 🧪 Step 3: Test Imports

In [ ]:
modules = [
    "from uvg_core.config import UVGConfig",
    "from uvg_core.script_generator import ScriptGenerator",
    "from uvg_core.tts_engine import TTSEngine",
    "from uvg_core.media_search import MediaSearchEngine",
    "from uvg_core.uvg_pipeline import UVGPipeline",
]

for m in modules:
    try:
        exec(m)
        print(f"✅ {m.split('import')[1].strip()}")
    except Exception as e:
        print(f"❌ {m.split('import')[1].strip()}: {e}")

## 🎬 Step 4: Generate Video

In [ ]:
from uvg_core.uvg_pipeline import UVGPipeline, PipelineConfig
import os

TOPIC = "The beauty of nature"
STYLE = "cinematic"

config = PipelineConfig(
    style_pack=STYLE,
    mock_mode=os.environ.get("UVG_MOCK_MODE", "true") == "true"
)

pipeline = UVGPipeline(config)
result = pipeline.run(topic=TOPIC)

print(f"{'✅ Success' if result.success else '❌ Failed'}: {result.output_path}")
if result.errors:
    print(f"Errors: {result.errors}")

## 🎥 Step 5: Watch

In [ ]:
from IPython.display import Video, display
import glob

videos = glob.glob("uvg_output/**/*.mp4", recursive=True)
if videos:
    display(Video(videos[0], embed=True, width=400))
else:
    print("No video found")